In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
#there are some irregular lines in the data, so setting argument to get a note of which lines and skip them when reading in
accre_gpu = pd.read_csv('../data/accre-gpu-jobs-2022-v2.csv', on_bad_lines='warn')

In [ ]:
#per our stakeholder, no need to include jobs with state of cancelled, pending, or running in our analysis, so dropping those 1258 rows
accre_gpu = accre_gpu.loc[(~accre_gpu['STATE'].isin(['PENDING', 'RUNNING'])) 
                          & 
                          (~accre_gpu['STATE'].str.contains('CANCELLED'))]

#stripping the 'M' from USEDMEM column and converting it to a numeric data type
accre_gpu['USEDMEM'] = accre_gpu['USEDMEM'].str.strip(to_strip='M').astype(float)

#converting REQTIME and USEDTIME columns to timedelta data types
accre_gpu['REQTIME'] = pd.to_timedelta(accre_gpu['REQTIME'].str.replace('-',' days '))
accre_gpu['USEDTIME'] = pd.to_timedelta(accre_gpu['USEDTIME'].str.replace('-',' days '))

#adding in a column that gives the difference in the time requested and time used
accre_gpu['time_dif'] = accre_gpu['REQTIME'] - accre_gpu['USEDTIME']

#adding in columns that provide versions of the other time columns but in minutes
accre_gpu['REQTIME_s'] = accre_gpu['REQTIME'].astype('timedelta64[s]')
accre_gpu['USEDTIME_s'] = accre_gpu['USEDTIME'].astype('timedelta64[s]')
accre_gpu['time_dif_s'] = accre_gpu['REQTIME_s'] - accre_gpu['USEDTIME_s']

accre_gpu = accre_gpu.reset_index(drop=True)

accre_gpu

#### 4. What is the distribution of different groups and users accessing each partition? In each partition, who are the top users, and do they represent a majority of the runtime-weighted jobs on the partition?   
From the stakeholder: For determining top users, I think the calculation would be the sum of nGPUs * usedtime over all their jobs in the partition. It would also be interesting to see what users/groups seem to be using more memory.

In [ ]:
#looking to see if users are associated with more than one account
#waiting to hear back if these actually are the same individuals, or if every account/user combo is unique
accre_gpu.groupby('USER')['ACCOUNT'].nunique().sort_values(ascending=False).head(25)

In [ ]:
#creating the RUNTIME weighted GPU value
accre_gpu['GPUS_RUNTIME'] = accre_gpu['GPUS'] * accre_gpu['USEDTIME_s']

#under the assumption that we want to analyze each user within the context of a specific account, creating a combined
#account_user column
accre_gpu['ACCOUNT_USER'] = accre_gpu['ACCOUNT'] + '_' + accre_gpu['USER']

accre_gpu

In [ ]:
#number of jobs either completed, failed, or ran out of memory for each user
user_counts = (
    accre_gpu
    .groupby(['PARTITION','ACCOUNT','USER'])['JOBID']
    .count()
    .to_frame()
    .reset_index()
    .sort_values('JOBID', ascending=False)
    .rename(columns={'JOBID':'job_count'})
)
user_counts

Almost half of all jobs in this data set came frome one user in one partition

In [ ]:
#distribution of users by total runtime weighted GPUs
sns.set(rc={"figure.figsize":(14, 10)})
sns.histplot(
    data=(accre_gpu
                    .groupby(['PARTITION', 'ACCOUNT_USER'])['GPUS_RUNTIME']
                    .sum()
                    .to_frame()
                    .reset_index()
                    .sort_values('GPUS_RUNTIME', ascending=False)),         
    x='GPUS_RUNTIME',
    hue='PARTITION',
    bins=27
);

In [ ]:
#distribution of users by total memory
sns.set(rc={"figure.figsize":(14, 10)})
sns.histplot(
    data=(accre_gpu
                    .groupby(['PARTITION', 'ACCOUNT_USER'])['USEDMEM']
                    .sum()
                    .to_frame()
                    .reset_index()
                    .sort_values('USEDMEM', ascending=False)),         
    x='USEDMEM',
    hue='PARTITION',
    bins=41
);

In [ ]:
#plotting the runtime weighted GPUs by partition and account
sns.set(rc={"figure.figsize":(10, 8)})
sns.barplot(
             x='GPUS_RUNTIME',
             y='ACCOUNT',
             hue='PARTITION',
             data=(accre_gpu
                    .groupby(['PARTITION', 'ACCOUNT'])['GPUS_RUNTIME']
                    .sum()
                    .to_frame()
                    .reset_index()
                    .sort_values('GPUS_RUNTIME', ascending=False))
            );

In [ ]:
#plotting runtime weighted GPUS by partition and user
#not yet sure if users can be under multiple accounts
sns.set(rc={"figure.figsize":(10,20)})
sns.barplot(
             x='GPUS_RUNTIME',
             y='ACCOUNT_USER',
             hue='PARTITION',
             data=(accre_gpu
                    .groupby(['PARTITION',
                              'ACCOUNT_USER'])['GPUS_RUNTIME']
                    .sum()
                    .to_frame()
                    .reset_index()
                    .sort_values('GPUS_RUNTIME', ascending=False)
                    .head(60)
                  )
);

In [ ]:
#plotting total memory usage by partition and account
sns.set(rc={"figure.figsize":(10, 8)})
sns.barplot(
             x='USEDMEM',
             y='ACCOUNT',
             hue='PARTITION',
             data=(accre_gpu
                    .groupby(['PARTITION', 'ACCOUNT'])['USEDMEM']
                    .sum()
                    .to_frame()
                    .reset_index()
                    .sort_values('USEDMEM', ascending=False))
            );

In [ ]:
#plotting total memory usage by partition and account_user
sns.set(rc={"figure.figsize":(10, 16)})
sns.barplot(
             x='USEDMEM',
             y='ACCOUNT_USER',
             hue='PARTITION',
             data=(accre_gpu
                    .groupby(['PARTITION', 'ACCOUNT_USER'])['USEDMEM']
                    .sum()
                    .to_frame()
                    .reset_index()
                    .sort_values('USEDMEM', ascending=False)
                    .head(60))
            );

In [ ]:
import plotly.express as px
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
#creating a dataframe of aggregated values for each user
user_details = (
    accre_gpu
    .groupby(['PARTITION', 'ACCOUNT', 'ACCOUNT_USER'])
    .agg(
        sum_GPUS = ('GPUS','sum'),
        sum_MEM = ('USEDMEM','sum'),
        sum_UTIME = ('USEDTIME', 'sum'),
        sum_RTIME = ('REQTIME', 'sum'),
        sum_time_dif = ('time_dif', 'sum'),
        sum_GPUS_RUNTIME = ('GPUS_RUNTIME', 'sum'),
        job_count = ('JOBID', 'count')
    )
    .reset_index()
)
user_details

In [ ]:
fig = px.scatter(user_details,
                 x="sum_GPUS_RUNTIME", 
                 y="sum_MEM", 
                 color="PARTITION",
                 size="job_count",
                 marginal_y="histogram",
                 marginal_x="histogram", 
                 template="simple_white",
                 height=700,
                 hover_data=["ACCOUNT", "ACCOUNT_USER", "sum_GPUS", "sum_UTIME", "job_count"])

fig.show()

In [ ]:
fig =  px.histogram(
    user_details, 
    x="sum_GPUS_RUNTIME",
    color="PARTITION",
    facet_row='PARTITION',
    height=750
)
    
fig.show()

In [ ]:
#@interact(partition = user_details['PARTITION'].unique())
#def interactive_plot(partition):

df = user_details

fig =  px.scatter(df, 
                  x="sum_GPUS_RUNTIME", 
                  y="sum_MEM",
                  color=(df["sum_time_dif"].astype('timedelta64[h]')),
                  size="job_count",
                  facet_row="PARTITION",
                  height=900,
                  width=600,
                  labels={
                      "sum_GPUS_RUNTIME" : "Total Runtime Weighted GPUs",
                      "sum_Mem" : "Total Memory Used (GB)",
                      "job_count" : "Total Jobs",
                      "sum_UTIME" : "Total Time Used",
                      "sum_GPUS" : "Total GPUs Requested"
                  },
                  #category_orders={"Year" : [2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014]},
                  hover_name="ACCOUNT_USER",
                  #hover_data={"Maternal_Death_Rate":True,
                  #            "Primary_ed_completion_percent":":.2f",
                  #            "GDP_Per_Capita":":$.2f", 
                  #            "Life_Expectancy":":.2f",
                  #            "Continent":True, 
                  #            "Year":False,
                  #            "Sex":False}
                 )

fig.update_layout(title_text="Title",
                  title_xanchor="auto",
                  title_font_size=14
                 #title_xref="paper"
                 )

fig.update_xaxes(title_font_size=12)

fig.update_yaxes(title_font_size=12)

fig.update_coloraxes(colorbar_len=1, 
                     colorbar_title_font_size=12,
                     colorbar=dict(orientation="h"),
                     colorbar_y=-.025,
                     colorbar_yanchor="top")

fig.show()